In [1]:
from datasets import load_dataset
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

In [2]:
dataset = load_dataset('cardiffnlp/tweet_topic_single')

Found cached dataset tweet_topic_single (C:/Users/schmi/.cache/huggingface/datasets/cardiffnlp___tweet_topic_single/tweet_topic_single/1.0.4/832eaa087889d9f4bc549869b44e0acb85a78364dfb3d2bc0bdf23a7224cf2ce)


  0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)),
])

text_clf.fit(dataset["train_all"]["text"], dataset["train_all"]["label"]) # type: ignore

predicted = text_clf.predict(dataset["test_2021"]["text"]) # type: ignore
print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7773183697578263
0.6055531424150178


In [4]:
text_clf_tfidf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)),
])

text_clf_tfidf.fit(dataset["train_all"]["text"], dataset["train_all"]["label"]) # type: ignore

predicted = text_clf_tfidf.predict(dataset["test_2021"]["text"]) # type: ignore
print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7454223272297696
0.46125052779952086


In [5]:
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)),
])

text_clf.fit(dataset["train_all"]["text"], dataset["train_all"]["label"]) # type: ignore

predicted = text_clf.predict(dataset["test_2021"]["text"]) # type: ignore
print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7666863555818074
0.5704291756162928


In [6]:
text_clf_tfidf = Pipeline([
    ('vect', CountVectorizer(binary=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)),
])

text_clf_tfidf.fit(dataset["train_all"]["text"], dataset["train_all"]["label"]) # type: ignore

predicted = text_clf_tfidf.predict(dataset["test_2021"]["text"]) # type: ignore
print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7566450088600118
0.4723803058224769


In [7]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__alpha': (1e-1, 1e-2, 1e-3, 1e-4),
}


In [8]:
gs_clf = GridSearchCV(text_clf_tfidf, parameters, cv=5, n_jobs=-1)

gs_clf = gs_clf.fit(dataset["train_all"]["text"], dataset["train_all"]["label"]) # type: ignore

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


predicted = gs_clf.predict(dataset["test_2021"]["text"]) # type: ignore
print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

clf__alpha: 0.0001
vect__ngram_range: (1, 1)
0.7885410513880685
0.6175367238607584
